In [3]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week2()

import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

--2020-08-29 23:42:55--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2020-08-29 23:42:55 (34.3 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]



In [1]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 3.8MB 55.5MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 512kB 54.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=f323d21c6883e091b3cbee8e41eda45973f9fac864d4a2ba3565de3b9df5e6f9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [88]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


tf.__version__

'1.15.0'

In [89]:
import re
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):

        line = line.strip()

        if not line:
            if tweet_tokens: # if theres any elements in list
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()

            # Replace all urls with <URL> token
            # Replace all users with <USR> token
            url = re.search(r'http', token)
            if url != None:
              token = '<URL>'
            
            user = re.search(r'@', token)
            if user != None:
              token = '<USR>'

            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [90]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [91]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [92]:
from collections import defaultdict

In [93]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    index = 0
    for i in special_tokens:
      tok2idx.update( { i : index } )
      index += 1


    
    #Create mappings from indices to tokens or tags
    special_set = set(special_tokens)
    idx2tok = []
    idx2tok += list(special_set)
    for i in tokens_or_tags:
      tokens_set = set(i)
      tokens_set = list(tokens_set.difference(special_set))
      idx2tok+=tokens_set
    

    idx2tok = list(set(idx2tok))
    
    special_len = len(tok2idx)

    index = 0
    for i in idx2tok:
      tok2idx.update( { i : index } )
      index += 1

  
    
    return tok2idx, idx2tok

In [94]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [95]:
print(token2idx)

print(all(list((map(lambda x: idx2tag.index(x)==tag2idx[x], idx2tag)))))

defaultdict(<function build_dict.<locals>.<lambda> at 0x7f28b8fb2400>, {'<UNK>': 6677, '<PAD>': 2163, '#GiveAdenanAChance': 0, 'weeding': 1, 'lightened': 2, 'Reason': 3, '30g': 4, 'drive': 5, 'Bodyswapping': 6, "post-'flash": 7, 'graders': 8, 'ugh': 9, 'D…': 10, 'expression': 11, 'count': 12, 'Loser': 13, 'Woo': 14, 'COMEBACK': 15, 'quelqun': 16, "life-don't": 17, 'silencieux': 18, 'Forecast-': 19, 'Jarred': 20, 'Shopping': 21, 'Classic': 22, 'Falls': 23, 'leav': 24, 'umbrella': 25, 'SOLO': 26, 'pleut': 27, 'Contact': 28, 'spent': 29, '/ht': 30, 'ooit': 31, '*throwing': 32, 'rescue': 33, '#4GElite2': 34, 'gods': 35, 'subs': 36, 'babes': 37, 'HALL': 38, 'transforming': 39, 'Seems': 40, 'Kenshin': 41, 'Taggart': 42, '#Chicago': 43, 'wait': 44, "f'ing": 45, 'ANO': 46, '0.97': 47, 'Construction': 48, 'Executive': 49, 'enhanced': 50, 'LAX': 51, 'What': 52, 'were': 53, '#sad': 54, 'kind': 55, 'BA': 56, '1.0152799119': 57, 'Sabre': 58, '#ColoradoSpringsShooting': 59, '#mito': 60, 'Demo': 61, 

In [96]:
len(idx2token), len(token2idx)


(20451, 20451)

In [97]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

# Batches generator

In [98]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):#for each batch:
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)# batch_end = n_samples for the last batch if allow smaller_last_batch=True
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))# mapping from words to index via token2idx
            y_list.append(tags2idxs(tags[idx]))# mapping from tags to index via tag2idx
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

In [99]:
dict1 = {}
list(map(lambda x :dict1.update({len(x):x}), train_tokens))
print(sorted(dict1.items())[-1]) # searching the longest sentence

(41, ['re', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 're', ':', 'Esther', 'Sikkimese', 'is', 'now', 'following', 'me', 'on', 'Twitter', '!', '<URL>', 'Thanks', 'a', 'bunch', '!', '103', 'January', '...'])


# Building model

In [101]:
class BiLSTMModel():
    pass

In [102]:
def declare_placeholders(self):
    """Specifies placeholders for the model."""

    # Placeholders for input and ground truth output.
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')#I had this as tf.string first
  
    # Placeholder for lengths of the sequences.
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    # Placeholder for a dropout keep probability. 
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    # Placeholder for a learning rate (tf.float32).
    self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[], name='learning_rate_ph')

In [103]:
# making the function above into a method of BiLSTMModel class
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [104]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """Specifies bi-LSTM architecture and computes logits for inputs."""
    
    # Create embedding variable (tf.Variable) with dtype tf.float32
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(initial_value=initial_embedding_matrix,
                                            dtype=tf.float32, name='embeddings_matrix')#""" !! I had a big trouble when I set the validate_shape=False here."""
    
    # Create RNN cells (for example, tf.nn.rnn_cell.BasicLSTMCell) with dropout wrapper
    forward_cell =  tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(n_hidden_rnn),
                                                  input_keep_prob = self.dropout_ph,
                                                  output_keep_prob=self.dropout_ph,
                                                  state_keep_prob=self.dropout_ph)
    backward_cell =  tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(n_hidden_rnn),
                                                   input_keep_prob = self.dropout_ph,
                                                  output_keep_prob=self.dropout_ph,
                                                  state_keep_prob=self.dropout_ph)

    # Look up embeddings for self.input_batch (tf.nn.embedding_lookup).
    embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
    
    # Pass them through Bidirectional Dynamic RNN (tf.nn.bidirectional_dynamic_rnn).
    (rnn_output_fw, rnn_output_bw), _ =  tf.nn.bidirectional_dynamic_rnn(
                                                                         cell_fw=forward_cell, 
                                                                         cell_bw=backward_cell,
                                                                         inputs=embeddings, 
                                                                         sequence_length=self.lengths,
                                                                         dtype=tf.float32)

    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # Dense layer on top.
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [105]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [106]:
def compute_predictions(self):
    """Transforms logits to probabilities and finds the most probable tags."""
    
    # Create softmax (tf.nn.softmax) function
    softmax_output = tf.nn.softmax(self.logits)
    
    # Use argmax (tf.argmax) to get the most probable tags
    self.predictions = tf.argmax(softmax_output, axis=-1)

In [107]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [108]:
def compute_loss(self, n_tags, PAD_index):
    """Computes masked cross-entopy loss with logits."""
    
    # Create cross entropy function function (tf.nn.softmax_cross_entropy_with_logits_v2)
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ground_truth_tags_one_hot, logits=self.logits)# I had this with the predictions first
    
    
    mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
    # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
    self.loss =  tf.reduce_mean(mask*loss_tensor)# I had this as tf.reduce_mean(tf.matmul(mask, loss_tensor), axis=1) and it did not work well at all

In [109]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

# Optimizer

In [110]:
def perform_optimization(self):
    """Specifies the optimizer and train_op for the model."""
    
    # Create an optimizer (tf.train.AdamOptimizer)
    self.optimizer =  tf.train.AdamOptimizer(self.learning_rate_ph)   #, beta=0.9, beta2=0.999, epsilon=1e-08, name='Adam')
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
    clip_norm = tf.cast(1.0, tf.float32)
    
    # self.grads_and_vars = [(tf.clip_by_norm(grads, clip_norm), vars) for grads, vars in self.grads_and_vars] at first I had self.grads_and_vars =  tf.clip_by_norm(self.grads_and_vars[0], clip_norm)
    self.grads_and_vars = [(tf.clip_by_norm(grad,clip_norm),var) for grad,var in self.grads_and_vars]
    print(self.grads_and_vars[:3])
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [111]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [112]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [113]:
BiLSTMModel.__init__ = classmethod(init_model)

# Training the network and predicting tags

In [114]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [115]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

In [116]:
def predict_for_batch(self, session, x_batch, lengths):
    feed_dict = {
        self.input_batch:x_batch, 
        self.lengths:lengths
        }

    predictions = session.run(self.predictions, feed_dict=feed_dict)
    
    return predictions

In [117]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

# elaluation

In [118]:
from evaluation import precision_recall_f1

In [119]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # Extending every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

In [127]:
from tensorflow.python.framework import ops
ops.reset_default_graph()


model = BiLSTMModel(vocabulary_size=20451, n_tags=len(tag2idx), embedding_dim=200, n_hidden_rnn=200, PAD_index= token2idx['<PAD>'])

batch_size = 32
n_epochs = 5
learning_rate = 0.01 #0.0005 was not very good

learning_rate_decay = np.sqrt(2.0)

dropout_keep_probability = 0.7 #0.1 was super shitty

print(model)

[(<tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7f28bae526a0>, <tf.Variable 'embeddings_matrix:0' shape=(20451, 200) dtype=float32_ref>), (<tf.Tensor 'clip_by_norm_1:0' shape=(400, 800) dtype=float32>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>), (<tf.Tensor 'clip_by_norm_2:0' shape=(800,) dtype=float32>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/bias:0' shape=(800,) dtype=float32_ref>)]


# Training


In [128]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 76143 phrases; correct: 181.

precision:  0.24%; recall:  4.03%; F1:  0.45

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 9338 phrases; correct: 14.

precision:  0.15%; recall:  2.61%; F1:  0.28

-------------------- Epoch 2 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 3609 phrases; correct: 1517.

precision:  42.03%; recall:  33.79%; F1:  37.47

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 294 phrases; correct: 132.

precision:  44.90%; recall:  24.58%; F1:  31.77

-------------------- Epoch 3 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4931 phrases; correct: 3490.

precision:  70.78%; recall:  77.75%; F1:  74.10

Validation data evaluation:
processed 12836 tokens with 537

In [124]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens, validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4568 phrases; correct: 4294.

precision:  94.00%; recall:  95.66%; F1:  94.82

	     company: precision:   94.85%; recall:   97.36%; F1:   96.09; predicted:   660

	    facility: precision:   92.19%; recall:   93.95%; F1:   93.06; predicted:   320

	     geo-loc: precision:   95.69%; recall:   97.99%; F1:   96.83; predicted:  1020

	       movie: precision:   81.01%; recall:   94.12%; F1:   87.07; predicted:    79

	 musicartist: precision:   90.34%; recall:   92.67%; F1:   91.49; predicted:   238

	       other: precision:   92.77%; recall:   94.98%; F1:   93.86; predicted:   775

	      person: precision:   97.11%; recall:   98.53%; F1:   97.82; predicted:   899

	     product: precision:   94.38%; recall:   94.97%; F1:   94.67; predicted:   320

	  sportsteam: precision:   91.47%; recall:   88.94%; F1:   90.19; predicted:   211

	      tvshow: precision:   67.39%; recall:  